In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from graphframes import GraphFrame
import os
#spark=SparkSession.builder.appName("graph").master("local[*]").getOrCreate()

spark = SparkSession.builder.appName("graphs")\
        .config('spark.jars.packages', 'io.graphframes:graphframes-spark4_2.13:0.10.0')\
        .master("local[*]")\
        .getOrCreate()

vertics = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])


# Edges DataFrame
edges = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(vertics, edges)
g.vertices.show()
g.edges.show()

:: loading settings :: url = jar:file:/home/pol/dev/zajecia/pyspark-graphs/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/pol/.ivy2.5.2/cache
The jars for the packages stored in: /home/pol/.ivy2.5.2/jars
io.graphframes#graphframes-spark4_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fbdf30f9-3c64-482f-aef3-9dbdb407451e;1.0
	confs: [default]
	found io.graphframes#graphframes-spark4_2.13;0.10.0 in central
	found io.graphframes#graphframes-graphx-spark4_2.13;0.10.0 in central
:: resolution report :: resolve 65ms :: artifacts dl 2ms
	:: modules in use:
	io.graphframes#graphframes-graphx-spark4_2.13;0.10.0 from central in [default]
	io.graphframes#graphframes-spark4_2.13;0.10.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       c

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
|  g|  e|      follow|
+---+---+------------+



In [4]:
from graphframes.classic.graphframe import StorageLevel
results = g.triangleCount(storage_level=StorageLevel.MEMORY_ONLY)
results.show()

26/01/12 17:04:54 WARN AggregateMessages: Returned DataFrame is persistent and materialized!
26/01/12 17:04:57 WARN TriangleCount$: Returned DataFrame is persistent and materialized!


+---+-------+---+----+------------+----+------------+-----+
| id|   name|age| src|src_triplets| dst|dst_triplets|count|
+---+-------+---+----+------------+----+------------+-----+
|  g|  Gabby| 60|NULL|        NULL|NULL|        NULL|    0|
|  f|  Fanny| 38|NULL|        NULL|NULL|        NULL|    0|
|  e| Esther| 32|NULL|        NULL|   e|           2|    1|
|  d|  David| 29|   d|           1|   d|           1|    1|
|  c|Charlie| 37|NULL|        NULL|NULL|        NULL|    0|
|  b|    Bob| 36|NULL|        NULL|NULL|        NULL|    0|
|  a|  Alice| 34|   a|           2|NULL|        NULL|    1|
+---+-------+---+----+------------+----+------------+-----+



In [ ]:
results = g.shortestPaths(landmarks=['a', 'c'])
results.show()